<a href="https://colab.research.google.com/github/GabeAspir/Patent-Prior-Art-Finder/blob/main/Interactive_patents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#@title Click the play button to begin, then continue to option 1 or 2
import pandas as pd
import gdown
import requests
import nltk
nltk.download('punkt')

# From https://changhsinlee.com/colab-import-python/
# If you are using GitHub, make sure you get the "Raw" version of the code
url = 'https://raw.githubusercontent.com/GabeAspir/Patent-Prior-Art-Finder/main/_DevFilesPatentPriorArtFinder.py'
r = requests.get(url)
 
# make sure your filename is the same as how you want to import 
with open('_DevFilesPatentPriorArtFinder.py', 'w') as f:
    f.write(r.text)
 
# now we can import
from _DevFilesPatentPriorArtFinder import _DevFilesPatentPriorArtFinder as paf

print("You may now proceed to option 1 or 2 below")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
You may now proceed to option 1 or 2 below


# Option 1: Compare an existing patent  

In [ ]:
#@title Enter a patent number for comparison
patent_Number = 'US-2018356996-A1' #@param {type:"string"}
print(patent_Number)
#pat= {"patent_Number": patent_Number}
# Need to validate the patent number
project_id = "semiotic-garden-315802"

pt = pd.io.gbq.read_gbq(f'''
  SELECT
    pub.publication_number as Publication_Number,
    ab.text as Abstract,
    STRING_AGG(citations.publication_number) as Citations
  FROM
    patents-public-data.patents.publications as pub,
    UNNEST (abstract_localized) AS ab,
    UNNEST (citation) as citations
  WHERE
    pub.publication_number IN ("{{}}")
  GROUP BY pub.publication_number, ab.text
  LIMIT
      1
'''.format(patent_Number), project_id=project_id)
pt= pt.iloc[0]


US-2018356996-A1
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=MsIs1b15RSV8UzH3e3J6OQzRRM7KW1&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWjN5rezm8KWNBIzYrpmnFB5_7LhFSMQsnCfRBzmXOqB1s_Obx71dW0


In [ ]:
#@title
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
print(pt)

  Publication_Number  ...                                          Citations
0   US-2018356996-A1  ...  US-2004143710-A1,US-2004193397-A1,US-201204731...

[1 rows x 3 columns]


# Option 2: Compare a new text

In [ ]:
import pandas as pd
new_text = 'A manufacturing process to produce a controllable integral membrane (28) in sheet-like photosensitive laminates (20), said photosensitive laminate being adapted to adhere to the surface to be etched (36). By the use of the present invention the revealed image (33) washes out without detail roots leaving the substrate (22), transfers easily, even if it is very fine, and can be etched on said surface to be etched very nicely.' #@param {type:"string"}
print(new_text)
new_cit = 'US-3916050-A,US-4371602-A,US-4430416-A,US-4511640-A,US-4587186-A,US-4716096-A,US-4764449-A,US-4801490-A'#@param {type:"string"}
print(new_cit)
pt= pd.Series([new_text,new_cit],index=["Abstract","Citations"])
print(pt)

A manufacturing process to produce a controllable integral membrane (28) in sheet-like photosensitive laminates (20), said photosensitive laminate being adapted to adhere to the surface to be etched (36). By the use of the present invention the revealed image (33) washes out without detail roots leaving the substrate (22), transfers easily, even if it is very fine, and can be etched on said surface to be etched very nicely.
US-3916050-A,US-4371602-A,US-4430416-A,US-4511640-A,US-4587186-A,US-4716096-A,US-4764449-A,US-4801490-A
Abstract     A manufacturing process to produce a controlla...
Citations    US-3916050-A,US-4371602-A,US-4430416-A,US-4511...
dtype: object


# Run the comparison
This will use a prepared set of 4 million patents. If you wish to load and train a new set for your comparison, click [here](https://colab.research.google.com/drive/1DrKOIpQIOMqTJ-jcAd0amhtBOXY9wQv3?usp=sharing).

In [ ]:
import gdown
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Load w2v Files
embed_files= ["1IOv-XTS5Q-FL7Wju09cE8nU65X6MmhKU",
              "1AUwg7UeBlCX0DN9AZM1URXl2cQXUmxeb"]
# TODO: Add dict and actual files
!mkdir -p "ppaf_files/emb"
directory= "ppaf_files"
for index in range(0,2):
  url = 'https://drive.google.com/uc?id='+ embed_files[index]
  output = 'ppaf_files/emb/file'+str(index)+'.json.gz'
  gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1IOv-XTS5Q-FL7Wju09cE8nU65X6MmhKU
To: /content/ppaf_files/emb/file0.json.gz
81.9MB [00:01, 61.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AUwg7UeBlCX0DN9AZM1URXl2cQXUmxeb
To: /content/ppaf_files/emb/file1.json.gz
81.9MB [00:01, 74.1MB/s]


# For the next step 
you must save the folders contained [here](https://drive.google.com/drive/folders/1TSjyNgCdvAIX92WF4BAMNeY5Ri4haANM?usp=sharing) In a folder of the root of your google drive called "ppaf_files". 
You should have a single folder ppaf_files that contians the other folders such as "meta" and "other"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
directory= "/content/drive/MyDrive/ppaf_files"

Use the check box to try running the comparison with and without tfidf (gives less frequent words more importance)

In [34]:
use_tfidf = False #@param {type:"boolean"}
artFinder = paf(directory)
print(artFinder.compareNewPatent(pt,directory,".9"))

Didn't make directories
Initialization complete T=2845.417324657


FileNotFoundError: ignored

In [ ]:
#@title Inspect a patent match
patent_Number = 'US-2018356996-A1' #@param {type:"string"}
print(patent_Number)

text = pd.io.gbq.read_gbq(f'''
  SELECT
    pub.publication_number as Publication_Number,
    ab.text as abstract_en,
    STRING_AGG(citations.publication_number) as Citations
  FROM
    patents-public-data.patents.publications as pub,
    UNNEST (abstract_localized) AS ab,
    UNNEST (citation) as citations
  WHERE
    pub.publication_number IN ("{{}}")
  GROUP BY pub.publication_number, ab.text
  LIMIT
      1
'''.format(patent_Number), project_id=project_id)
print(text["abstract_en"][0])

US-2018356996-A1
A storage system having an adaptive workload-based command processing clock is provided. In one embodiment, a storage system has a memory, a command processing path, and a controller in communication with the memory and the command processing path. The controller is configured to adapt an input clock signal based on a current workload of the controller and provide the adapted clock signal to the command processing path in the controller.
